In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

#open window
driver = webdriver.Chrome()

driver.get("https://writer.com/ai-content-detector/")
driver.maximize_window()

#locate elements
elem = driver.find_element(By.CLASS_NAME, "ai_textbox")

#imp functions

# function for removing html tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')  # Compile the regex pattern for HTML tags
     # Remove HTML tags
    text = re.sub(clean, '', text)
    # Replace underscores with empty spaces
    text = text.replace('_', ' ')
    return text

# Main scrapping function
def ai_text_score(text):
    elem.clear()
    elem.send_keys(text)
    submit = driver.find_element(By.CLASS_NAME, "ai-content-detector-submit")
    submit.click()
    classes = submit.get_attribute("class")

    while "dc-btn-gradient_loading" in classes: # waits till result is loaded
        time.sleep(1)
        classes = submit.get_attribute("class")
    res = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ai-percentage")))
    return res.text

#load dataframe and clean it
df = pd.read_excel('./Results.xlsx')
df = df.dropna(axis = 0 , ignore_index=False , subset= 'Abstract')
df['Abstract'] = df["Abstract"].apply(remove_html_tags)

last_index = 150 #last index till the loop will run

try:
    for ind in df.index:
        if pd.isnull(df['Writer'][ind]):
            print("runs")
            if len(df['Abstract'][ind].split()) < 60 :
                df['Writer'][ind] = 0
                continue
            df['Writer'][ind]= 100 - int(ai_text_score(df['Abstract'][ind]))
        if ind == last_index:
            break
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
     driver.close()
df.to_excel("Results.xlsx", index = False)
print(df)